In [26]:
from abc import ABC, abstractmethod


# Singleton
class SingletonMeta(type):
    _instâncias = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instâncias:
            instância = super().__call__(*args, **kwargs)
            cls._instâncias[cls] = instância
        return cls._instâncias[cls]


class Elevador:
    def __init__(self):
        self.estado = Parar()
        self.observadores = []
        self.velocidade = 1.0

    def mudar_estado(self, novo_estado):
        saindo_manutencao = isinstance(self.estado, Manutencao)
        self.estado = novo_estado
        self.estado.executar(self)

        if isinstance(novo_estado, Manutencao):
            self.notificar_observadores_manutencao()
        elif saindo_manutencao:
            self.notificar_observadores_saindo_manutencao()

    def adicionar_observador(self, observador):
        self.observadores.append(observador)

    def remover_observador(self, observador):
        self.observadores.remove(observador)

    def notificar_observadores_emperramento(self):
        for observador in self.observadores:
            observador.notificar_emperramento(self)

    def notificar_observadores_manutencao(self):
        for observador in self.observadores:
            observador.notificar_manutencao(self)

    def notificar_observadores_saindo_manutencao(self):
        for observador in self.observadores:
            if hasattr(observador, "notificar_saindo_manutencao"):
                observador.notificar_saindo_manutencao(self)


# Factory
class ElevadorFactory:
    @staticmethod
    def criar_elevador(tipo):
        if tipo == "A":
            return ElevadorA()
        elif tipo == "B":
            return ElevadorB()
        else:
            raise ValueError(f"Tipo de elevador inválido: {tipo}")


class ElevadorA(Elevador):
    def __init__(self):
        super().__init__()
        self.nome = "Elevador A"

    def restaurar_velocidade(self):
        self.velocidade = 1.0


class ElevadorB(Elevador):
    def __init__(self):
        super().__init__()
        self.nome = "Elevador B"

    def restaurar_velocidade(self):
        self.velocidade = 1.0


# State
class EstadoElevador(ABC):
    @abstractmethod
    def executar(self, elevador):
        pass


class Subir(EstadoElevador):
    def executar(self, elevador):
        print("O elevador está subindo.")


class Descer(EstadoElevador):
    def executar(self, elevador):
        print("O elevador está descendo.")


class Parar(EstadoElevador):
    def executar(self, elevador):
        print("O elevador está parado.")


class Emperrado(EstadoElevador):
    def executar(self, elevador):
        print("O elevador está emperrado.")
        elevador.notificar_observadores_emperramento()


class Manutencao(EstadoElevador):
    def executar(self, elevador):
        print("O elevador está em manutenção.")


# Observer
class Observador(ABC):
    @abstractmethod
    def notificar_emperramento(self, elevador):
        pass

    def notificar_manutencao(self, elevador):
        pass


class EquipeManutencao(Observador):
    def notificar_emperramento(self, elevador):
        print(f"Equipe de manutenção notificada: {elevador} emperrado.")
    
    def notificar_manutencao(self, elevador):
        pass  # Não faz nada ao ser notificado sobre manutenção


class EquipeSeguranca(Observador):
    def notificar_emperramento(self, elevador):
        print(f"Equipe de segurança notificada: {elevador} emperrado.")

    def notificar_manutencao(self, elevador):
        pass


class MusicaAgradavel(Observador):
    def notificar_emperramento(self, elevador):
        print(f"Tocando música agradável no {elevador} enquanto ele está emperrado.")

    def notificar_manutencao(self, elevador):
        pass


class RegulaVelocidade(Observador):
    def __init__(self, elevadores):
        self.elevadores = elevadores

    def notificar_emperramento(self, elevador):
        pass

    def notificar_manutencao(self, elevador):
        elevador_em_manutencao = any(isinstance(e.estado, Manutencao) for e in self.elevadores)
        if elevador_em_manutencao:
            for e in self.elevadores:
                if e != elevador:
                    e.velocidade = 1.5
                    print(f"A velocidade do {e} foi aumentada para {e.velocidade}")
        else:
            for e in self.elevadores:
                e.velocidade = 1.0
                print(f"A velocidade do {e} foi restaurada para {e.velocidade}") 

    def notificar_saindo_manutencao(self, elevador):
        if elevador in self.elevadores:
            for outro_elevador in self.elevadores:
                if outro_elevador != elevador:
                    outro_elevador.restaurar_velocidade()
                    print(f"A velocidade do {outro_elevador} foi restaurada para {outro_elevador.velocidade}")


def main():
    elevador_a = ElevadorFactory.criar_elevador("A")
    elevador_b = ElevadorFactory.criar_elevador("B")

    equipe_manutencao = EquipeManutencao()
    equipe_seguranca = EquipeSeguranca()
    musica_agradavel = MusicaAgradavel()
    regula_velocidade = RegulaVelocidade([elevador_a, elevador_b])


    elevador_a.adicionar_observador(equipe_manutencao)
    elevador_a.adicionar_observador(equipe_seguranca)
    elevador_a.adicionar_observador(musica_agradavel)
    elevador_a.adicionar_observador(regula_velocidade)

    elevador_b.adicionar_observador(equipe_manutencao)
    elevador_b.adicionar_observador(equipe_seguranca)
    elevador_b.adicionar_observador(musica_agradavel)
    elevador_b.adicionar_observador(regula_velocidade)

    print("\n-- Elevador A sobe --")
    elevador_a.mudar_estado(Subir())

    print("\n-- Elevador A emperrado --")
    elevador_a.mudar_estado(Emperrado())

    print("\n-- Elevador A entra em manutenção --")
    elevador_a.mudar_estado(Manutencao())

    print("\n-- Elevador A entra desce --")
    elevador_a.mudar_estado(Descer())

    print("\n-- Elevador B para --")
    elevador_b.mudar_estado(Parar())

    print("\n-- Elevador B emperrado --")
    elevador_b.mudar_estado(Emperrado())

    print("\n-- Elevador B entra em manutenção --")
    elevador_b.mudar_estado(Manutencao())

if __name__ == "__main__":
    main()



-- Elevador A sobe --
O elevador está subindo.

-- Elevador A emperrado --
O elevador está emperrado.
Equipe de manutenção notificada: <__main__.ElevadorA object at 0x7ff04f385870> emperrado.
Equipe de segurança notificada: <__main__.ElevadorA object at 0x7ff04f385870> emperrado.
Tocando música agradável no <__main__.ElevadorA object at 0x7ff04f385870> enquanto ele está emperrado.

-- Elevador A entra em manutenção --
O elevador está em manutenção.
A velocidade do <__main__.ElevadorB object at 0x7ff04f3863b0> foi aumentada para 1.5

-- Elevador A entra desce --
O elevador está descendo.
A velocidade do <__main__.ElevadorB object at 0x7ff04f3863b0> foi restaurada para 1.0

-- Elevador B para --
O elevador está parado.

-- Elevador B emperrado --
O elevador está emperrado.
Equipe de manutenção notificada: <__main__.ElevadorB object at 0x7ff04f3863b0> emperrado.
Equipe de segurança notificada: <__main__.ElevadorB object at 0x7ff04f3863b0> emperrado.
Tocando música agradável no <__main__.

Neste código, temos várias classes e padrões de projeto. As classes e seus relacionamentos são explicadas a seguir:

1. `SingletonMeta`: Esta classe é um metaclass que implementa o padrão Singleton, garantindo que cada classe que utiliza esta metaclass tenha apenas uma instância.

2. `Elevador`: Classe base para os elevadores. Esta classe tem uma instância de um estado (uma subclasse de `EstadoElevador`) chamado `estado`, uma lista de observadores chamada `observadores` e um atributo `velocidade`. Há também métodos para mudar o estado, adicionar e remover observadores, e notificar os observadores em diferentes situações.

3. `ElevadorA` e `ElevadorB`: Subclasses de `Elevador`. Ambas têm um atributo `nome` e um método `restaurar_velocidade`.

4. `ElevadorFactory`: Uma classe factory que cria instâncias de `ElevadorA` ou `ElevadorB` com base no tipo de elevador especificado.

5. `EstadoElevador`: Classe abstrata para os estados do elevador. Tem um método abstrato chamado `executar`.

6. `Subir`, `Descer`, `Parar`, `Emperrado` e `Manutencao`: Subclasses de `EstadoElevador`. Cada uma implementa o método `executar` de acordo com seu comportamento específico.

7. `Observador`: Classe abstrata para os observadores dos elevadores. Tem dois métodos abstratos: `notificar_emperramento` e `notificar_manutencao`.

8. `EquipeManutencao`, `EquipeSeguranca`, `MusicaAgradavel` e `RegulaVelocidade`: Subclasses de `Observador`. Cada uma implementa os métodos `notificar_emperramento` e `notificar_manutencao` de acordo com seu comportamento específico. Além disso, a classe `RegulaVelocidade` também implementa um método `notificar_saindo_manutencao`.

No geral, temos as seguintes relações entre as classes:

- `ElevadorA` e `ElevadorB` herdam de `Elevador`.
- `Subir`, `Descer`, `Parar`, `Emperrado` e `Manutencao` herdam de `EstadoElevador`.
- `EquipeManutencao`, `EquipeSeguranca`, `MusicaAgradavel` e `RegulaVelocidade` herdam de `Observador`.
- `Elevador` tem uma associação com `EstadoElevador` e `Observador`.
- `RegulaVelocidade` tem uma associação com `Elevador`.

O código também utiliza os padrões de projeto Singleton, Factory, State e Observer:

- Singleton: implementado na metaclass `SingletonMeta`, que é aplicado às classes que devem ter apenas uma instância.
- Factory: implementado na classe `ElevadorFactory`, que cria instâncias de elevadores do tipo especificado.
- State: implementado nas classes `EstadoElevador`, `Subir`, `Descer`, `Parar`, `Emperrado` e `Manutencao`, que representam os diferentes estados de um elevador.
- Observer: implementado nas classes `Observador`, `EquipeManutencao`, `EquipeSeguranca`, `MusicaAgradavel` e `RegulaVelocidade`, que representam diferentes entidades que reagem às mudanças de estado dos elevadores. Os elevadores, por sua vez, notificam seus observadores registrados sempre que ocorre uma mudança relevante no estado, como emperramento ou manutenção.

As instâncias das classes de observadores são adicionadas à lista de observadores do elevador e, dependendo das mudanças de estado do elevador, os observadores são notificados e executam suas ações correspondentes.

Neste sistema, a classe `RegulaVelocidade` é um observador especial que ajusta a velocidade dos outros elevadores sempre que um elevador entra ou sai do estado de manutenção. Quando um elevador entra em manutenção, a velocidade dos outros elevadores aumenta, e quando um elevador sai da manutenção, a velocidade dos outros elevadores retorna ao normal.

Em resumo, este código implementa um sistema de elevadores que utiliza os padrões de projeto Singleton, Factory, State e Observer para gerenciar seus estados e notificar diferentes entidades quando eventos relevantes ocorrem. Além disso, a velocidade dos elevadores é ajustada com base no estado de manutenção dos outros elevadores no sistema.